In [26]:
import math

def return_flow_rate(path, rates):
    flow_rate = 0
    activated = []
    for j in range(len(path)-1):
        if path[j] == path[j+1] and path[j] not in activated and rates[path[j]] != 0:
            activated.append(path[j])
            flow_rate += (24-j) * rates[path[j]]
    return flow_rate, activated

def return_upper_bound(i, last1, last2, activated1, activated2, rates):
    not_selected_rates = []
    for node in rates:
        if node not in activated1 and node not in activated2 and rates[node] != 0:
            not_selected_rates.append(rates[node])
    not_selected_rates.sort(reverse=True)
    path_a = []
    path_b = []
    upper_bound = 0

    upper_bound += (rates[last1] + rates[last2]) * (27-i)
    for j in range(len(not_selected_rates)):
        if j == 0 or j%2:
            path_a.append(not_selected_rates[j])
            continue
        else:
            path_b.append(not_selected_rates[j])
            continue
    
    for j in range(len(path_a)-1):
        upper_bound += (27-i-2*j) * path_a[j]
    for j in range(len(path_b)-1):
        upper_bound += (27-i-2*j) * path_b[j]
    return upper_bound


rates = {}
links = {}
nodes = []

nonzero_valves = 0

try:
    f = open("input", encoding = 'utf-8')
    for row in f:
        line = row.strip().split(" ")
        valve = line[1]
        nodes.append(valve)
        rate = int(line[4][:-1].split("=")[1])
        if rate > 0:
           nonzero_valves += 1 
        rates[valve] = rate
        exits = line[9:]
        link = [valve]
        for exit in exits:
            if exit[-1] == ',':
                link.append(exit[0:-1])
            else:
                link.append(exit)
        links[valve] = link

finally:
    f.close()

highest_flow_rate = -1
paths_1 = []
paths_2 = []

no_of_exits = len(links['AA'])
for no in range(no_of_exits):
    new_path = []
    new_path.append(links['AA'][no])
    paths_1.append(new_path.copy())
    paths_2.append(new_path.copy())

for i in range(1, 26):
    new_paths = []
    for path in paths_1:
        no_of_exits = len(links[path[-1]])
        for no in range(no_of_exits):
            # mechanism 0
            if len(path) >= 2 and path[-2] == path[-1] and path[-1] == links[path[-1]][no]:
                continue
            # mechanism 1
            if len(path) >= 2 and path[-2] == links[path[-1]][no] and path[-1] != path[-2]:
                continue
            # mechanism 2
            allowed = True
            if links[path[-1]][no] in path:
                allowed = False
                pos = len(path) - 1 - path[::-1].index(links[path[-1]][no])
                if pos+1 == len(path):
                    allowed = True
                else:
                    for j in range(pos, len(path)-1):
                        if path[j] == path[j+1]:
                            allowed = True
                            break
            if allowed is False:
                continue
            # mechanism 3
            allowed = True
            if len(path) >= 4 and path[-1] == links[path[-1]][no]:
                for j in range(len(path)-2):
                    if path[j] == path[j+1] and path[j] == links[path[-1]][no] and path[j+1] != path[j+2]:
                        allowed = False
                        break
            if allowed is False:
                continue
            new_path = path.copy()
            new_path.append(links[path[-1]][no])
            new_paths.append(new_path)
    
    paths_1 = new_paths

    new_paths = []
    for path in paths_2:
        no_of_exits = len(links[path[-1]])
        for no in range(no_of_exits):
            # mechanism 0
            if len(path) >= 2 and path[-2] == path[-1] and path[-1] == links[path[-1]][no]:
                continue
            # mechanism 1
            if len(path) >= 2 and path[-2] == links[path[-1]][no] and path[-1] != path[-2]:
                continue
            # mechanism 2
            allowed = True
            if links[path[-1]][no] in path:
                allowed = False
                pos = len(path) - 1 - path[::-1].index(links[path[-1]][no])
                if pos+1 == len(path):
                    allowed = True
                else:
                    for j in range(pos, len(path)-1):
                        if path[j] == path[j+1]:
                            allowed = True
                            break
            if allowed is False:
                continue
            # mechanism 3
            allowed = True
            if len(path) >= 4 and path[-1] == links[path[-1]][no]:
                for j in range(len(path)-2):
                    if path[j] == path[j+1] and path[j] == links[path[-1]][no] and path[j+1] != path[j+2]:
                        allowed = False
                        break
            if allowed is False:
                continue
            new_path = path.copy()
            new_path.append(links[path[-1]][no])
            new_paths.append(new_path)

    paths_2 = new_paths
    unique_p1 = {}
    unique_p2 = {}
    activated = []
    for path1 in paths_1:
        flow_rate1, activated1 = return_flow_rate(path1, rates)
        activated1.sort()
        act1set = set(activated1)
        for path2 in paths_2:
            flow_rate2, activated2 = return_flow_rate(path2, rates)
            if not act1set.isdisjoint(activated2):
                continue
            activated2.sort()

            flow_rate = flow_rate1 + flow_rate2
            if flow_rate > highest_flow_rate:
                highest_flow_rate = flow_rate

            temp_key1 = str(activated1) + str(path1[-1])
            temp_key2 = str(activated2) + str(path2[-1])
            no_of_activated = len(activated1) + len(activated2)
            upper_bound = 0
            if no_of_activated != nonzero_valves:
                upper_bound = return_upper_bound(i, path1[-1], path2[-1], activated1, activated2, rates)
            if no_of_activated != nonzero_valves and flow_rate + upper_bound > highest_flow_rate:
                if temp_key1 not in unique_p1:
                    unique_p1[temp_key1] = [path1, flow_rate1]
                else:
                    if unique_p1[temp_key1][1] < flow_rate1:
                        unique_p1[temp_key1] = [path1, flow_rate1]
                if temp_key2 not in unique_p2:
                    unique_p2[temp_key2] = [path2, flow_rate2]
                else:
                    if unique_p2[temp_key2][1] < flow_rate2:
                        unique_p2[temp_key2] = [path2, flow_rate2] 
            else:
                if temp_key1 in unique_p1:
                    del unique_p1[temp_key1]
                if temp_key2 in unique_p2:
                    del unique_p2[temp_key2]

    paths_1 = []
    paths_2 = []
    for key in unique_p1:
        paths_1.append(unique_p1[key][0])
    for key in unique_p2:
        paths_2.append(unique_p2[key][0])

print("Highest flow rate:", highest_flow_rate)


1 0:00:00.001088
2 0:00:00.007041
3 0:00:00.018407
4 0:00:00.078815
5 0:00:00.243240
6 0:00:00.691413
7 0:00:01.968455
8 0:00:04.869964
9 0:00:10.670799
10 0:00:24.291165
11 0:00:53.162320
12 0:01:48.485872
13 0:03:33.889534
14 0:07:06.591907
15 0:13:38.561604
16 0:22:48.243772
17 0:04:48.696100
18 0:03:34.123148
19 0:01:34.125280
20 0:01:13.591453
21 0:00:27.401859
22 0:00:22.083650
23 0:00:03.396026
24 0:00:00.086233
25 0:00:00.004619
Highest flow rate: 3015
